In [1]:
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !jupyter nbextension enable --py --sys-prefix qgrid


# so we can call the g2p service
import requests
import json

# for processing results
import pandas as pd
import numpy as np
from collections import Counter
import datetime


# for displays and interactivity
import matplotlib.pyplot as plt
import qgrid
from IPython.display import IFrame    
from IPython.display import Javascript
from IPython.display import display, Markdown  



 


In [2]:
# 

def to_df(generator, feature ):
    """ some client side utility code to take the query results and xform it into a data frame """   
    def to_dict(generator, feature):
        for hit in generator:
            drugs = [ec.get('term', ec['description']).encode('utf-8') for ec in hit['association']['environmentalContexts']]
            description = hit['association']['description'].encode('utf-8')
            source = hit['source'].encode('utf-8')
            evidence_label = hit['association']['evidence_label']
            id = hit['evidence.id']
            phenotype = hit['association']['phenotype']['type']['term'].encode('utf-8')
            publications = []
            matches = []
            
            
            try:
                # compare the queried feature vs all the features in the hit.  highlight the matches
                f = feature
                if f:
                    for f2 in hit.get('features', []):
                        if not f2:
                            continue
                        for s in f.get('synonyms', []):
                            if s in f2.get('synonyms', []):
                                matches.append(s)
                        for p in f.get('pathways',[]):
                            if p in f2.get('pathways',[]):
                                # matches.append(p)
                                matches.append('pathway')
                        for sp in f.get('swissprots',[]):
                            if sp in f2.get('swissprots',[]):
                                matches.append(sp)

                        if f.get('sequence_ontology', {'name':'X'}) == f2.get('sequence_ontology', {'name':None}):
                            matches.append(f['sequence_ontology']['name'])

                        if f.get('geneSymbol', None) == f2.get('geneSymbol', None):
                            matches.append(f.get('geneSymbol', None))
                            
                matches = list(set(matches))
                
                for e in hit['association']['evidence']:
                    for p in e['info']['publications']:
                        publications.append(p)


                yield {'id':id,
                       'source': source,
                       'evidence_label': evidence_label,
                       'drugs': drugs,
                       'description': description,
                       'phenotype':phenotype,
                       'publications':publications,
                       'matches': matches
                      }
            except Exception as e:
                print '!!!',e
                
    
    try:        
        return pd.DataFrame.from_records( to_dict(generator, feature) , index='id')
    except Exception as e:
        # might be empty
        return pd.DataFrame()
    


In [20]:
# chohort / patient's queries
# g2p_url = 'http://10.241.39.211:8080/api/v1/features/associations'
g2p_url = 'https://dms-dev.compbio.ohsu.edu/api/v1/features/associations'
# p115 = [{'description': '', 'start': 27100988L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ARID1A', 'alt': 'T', 'ref': 'C', 'chromosome': '1', 'name': 'ARID1A '}, {'description': '', 'start': 204494718L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'MDM4', 'alt': 'G', 'ref': 'C', 'chromosome': '1', 'name': 'MDM4 '}, {'description': '', 'start': 30143341L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ALK', 'alt': 'A', 'ref': 'G', 'chromosome': '2', 'name': 'ALK '}, {'description': '', 'start': 48028096L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'MSH6', 'alt': 'C', 'ref': 'G', 'chromosome': '2', 'name': 'MSH6 '}, {'description': '', 'start': 10084764L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'FANCD2', 'alt': 'G', 'ref': 'C', 'chromosome': '3', 'name': 'FANCD2 '}, {'description': '', 'start': 142185372L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ATR', 'alt': 'G', 'ref': 'C', 'chromosome': '3', 'name': 'ATR '}, {'description': '', 'start': 178916923L, 'biomarker_type': 'indel', 'referenceName': 'GRCh37', 'geneSymbol': 'PIK3CA', 'alt': 'C', 'ref': 'CCAGTAG', 'chromosome': '3', 'name': 'PIK3CA '}, {'description': '', 'start': 117647566L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ROS1', 'alt': 'C', 'ref': 'G', 'chromosome': '6', 'name': 'ROS1 '}, {'description': '', 'start': 87285753L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'NTRK2', 'alt': 'G', 'ref': 'C', 'chromosome': '9', 'name': 'NTRK2 '}, {'description': '', 'start': 108106483L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'ATM', 'alt': 'C', 'ref': 'G', 'chromosome': '11', 'name': 'ATM '}, {'description': '', 'start': 45618180L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'FANCM', 'alt': 'G', 'ref': 'C', 'chromosome': '14', 'name': 'FANCM '}, {'description': '', 'start': 45644454L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'FANCM', 'alt': 'C', 'ref': 'G', 'chromosome': '14', 'name': 'FANCM '}, {'description': '', 'start': 2110733L, 'biomarker_type': 'substitution', 'referenceName': 'GRCh37', 'geneSymbol': 'TSC2', 'alt': 'G', 'ref': 'C', 'chromosome': '16', 'name': 'TSC2 '}, {'description': 'GAIN Copy Number Ratio 3.0528', 'start': 39771328L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'IDO1', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'IDO1 '}, {'description': 'GAIN Copy Number Ratio 5.3306', 'start': 39792474L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'IDO2', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'IDO2 '}, {'description': 'LOSS Copy Number Ratio 0.3497', 'start': 21967751L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'CDKN2A', 'alt': '', 'ref': '', 'chromosome': '9', 'name': 'CDKN2A '}, {'description': 'GAIN Copy Number Ratio 7.5539', 'start': 38268656L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'FGFR1', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'FGFR1 '}, {'description': 'LOSS Copy Number Ratio 0.5662', 'start': 89623195L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'PTEN', 'alt': '', 'ref': '', 'chromosome': '10', 'name': 'PTEN '}, {'description': 'GAIN Copy Number Ratio 2.883', 'start': 69455873L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'CCND1', 'alt': '', 'ref': '', 'chromosome': '11', 'name': 'CCND1 '}, {'description': 'GAIN Copy Number Ratio 2.0415', 'start': 128748315L, 'biomarker_type': 'copy_number_variation', 'referenceName': 'GRCh37', 'geneSymbol': 'MYC', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'MYC '}]  # NOQA
# p204 = [{'end': 41266125L, 'name': 'CTNNB1 p.T41I', 'start': 41266125L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CTNNB1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'catenin (cadherin-associated protein), beta 1, 88kDa'}, {'end': 142188239L, 'name': 'ATR p.I2164M', 'start': 142188239L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ATR', 'alt': 'C', 'ref': 'T', 'chromosome': '3', 'description': 'ATR serine/threonine kinase'}, {'end': 89685310L, 'name': 'PTEN p.-70fs', 'start': 89685309L, 'biomarker_type': 'Frame_Shift_Ins', 'referenceName': 'GRCh37', 'geneSymbol': 'PTEN', 'alt': 'AATCT', 'ref': '-', 'chromosome': '10', 'description': 'phosphatase and tensin homolog'}, {'end': 41266125L, 'name': 'CTNNB1 p.T41I', 'start': 41266125L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CTNNB1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'catenin (cadherin-associated protein), beta 1, 88kDa'}, {'end': 1584977L, 'name': 'P2RY8 p.A159T', 'start': 1584977L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'P2RY8', 'alt': 'T', 'ref': 'C', 'chromosome': 'X', 'description': 'purinergic receptor P2Y, G-protein coupled, 8'}, {'end': 170802031L, 'name': 'TNIK p.T1028A', 'start': 170802031L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'TNIK', 'alt': 'C', 'ref': 'T', 'chromosome': '3', 'description': 'TRAF2 and NCK interacting kinase'}, {'end': 118764345L, 'name': 'CXCR5 p.S31F', 'start': 118764345L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CXCR5', 'alt': 'T', 'ref': 'C', 'chromosome': '11', 'description': 'chemokine (C-X-C motif) receptor 5'}, {'end': 136071095L, 'name': 'ZRANB3 p.L310F', 'start': 136071095L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ZRANB3', 'alt': 'G', 'ref': 'C', 'chromosome': '2', 'description': 'zinc finger, RAN-binding domain containing 3'}, {'end': 228566043L, 'name': 'OBSCN p.V8839A', 'start': 228566043L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'OBSCN', 'alt': 'C', 'ref': 'T', 'chromosome': '1', 'description': 'obscurin, cytoskeletal calmodulin and titin-interacting RhoGEF'}, {'end': 2097401L, 'name': 'STK35 p.L328V', 'start': 2097401L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'STK35', 'alt': 'G', 'ref': 'C', 'chromosome': '20', 'description': 'serine/threonine kinase 35'}, {'end': 58130871L, 'name': 'AGAP2 p.A387T', 'start': 58130871L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'AGAP2', 'alt': 'T', 'ref': 'C', 'chromosome': '12', 'description': 'ArfGAP with GTPase domain, ankyrin repeat and PH domain 2'}, {'end': 33690771L, 'name': 'IP6K3 p.R320H', 'start': 33690771L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'IP6K3', 'alt': 'T', 'ref': 'C', 'chromosome': '6', 'description': 'inositol hexakisphosphate kinase 3'}, {'end': 101910837L, 'name': 'NALCN p.G408A', 'start': 101910837L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'NALCN', 'alt': 'G', 'ref': 'C', 'chromosome': '13', 'description': 'sodium leak channel, non-selective'}, {'end': 57424051L, 'name': 'MYO1A p.R845K', 'start': 57424051L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'MYO1A', 'alt': 'T', 'ref': 'C', 'chromosome': '12', 'description': 'myosin IA'}, {'end': 38211533L, 'name': 'TRPC4 p.R814K', 'start': 38211533L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'TRPC4', 'alt': 'T', 'ref': 'C', 'chromosome': '13', 'description': 'transient receptor potential cation channel, subfamily C, member 4'}, {'end': 32257834L, 'name': 'SPOCD1 p.R982C', 'start': 32257834L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'SPOCD1', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'SPOC domain containing 1'}, {'end': 28097282L, 'name': 'ZSCAN16 p.E201K', 'start': 28097282L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ZSCAN16', 'alt': 'A', 'ref': 'G', 'chromosome': '6', 'description': 'zinc finger and SCAN domain containing 16'}, {'end': 84628811L, 'name': 'SEMA3D p.R760Q', 'start': 84628811L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'SEMA3D', 'alt': 'T', 'ref': 'C', 'chromosome': '7', 'description': 'sema domain, immunoglobulin domain (Ig), short basic domain, secreted, (semaphorin) 3D'}, {'end': 72400932L, 'name': 'NEGR1 p.A80V', 'start': 72400932L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'NEGR1', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'neuronal growth regulator 1'}, {'end': 35778772L, 'name': 'ARPP21 p.P521L', 'start': 35778772L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ARPP21', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'cAMP-regulated phosphoprotein, 21kDa'}, {'end': 21016738L, 'name': 'KIF17 p.R442S', 'start': 21016738L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'KIF17', 'alt': 'T', 'ref': 'G', 'chromosome': '1', 'description': 'kinesin family member 17'}, {'end': 51929267L, 'name': 'IQCF1 p.R86Q', 'start': 51929267L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'IQCF1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'IQ motif containing F1'}, {'end': 95188799L, 'name': 'CDH17 p.E132K', 'start': 95188799L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CDH17', 'alt': 'T', 'ref': 'C', 'chromosome': '8', 'description': 'cadherin 17, LI cadherin (liver-intestine)'}, {'end': 96117712L, 'name': 'CCDC82 p.L67P', 'start': 96117712L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CCDC82', 'alt': 'G', 'ref': 'A', 'chromosome': '11', 'description': 'coiled-coil domain containing 82'}, {'end': 105967614L, 'name': 'AASDHPPT p.R304*', 'start': 105967614L, 'biomarker_type': 'Nonsense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'AASDHPPT', 'alt': 'T', 'ref': 'C', 'chromosome': '11', 'description': 'aminoadipate-semialdehyde dehydrogenase-phosphopantetheinyl transferase'}, {'end': 31538698L, 'name': 'CLDN17 p.A80S', 'start': 31538698L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CLDN17', 'alt': 'A', 'ref': 'C', 'chromosome': '21', 'description': 'claudin 17'}, {'end': 38483424L, 'name': 'UTP11L p.M70I', 'start': 38483424L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'UTP11L', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'UTP11-like, U3 small nucleolar ribonucleoprotein (yeast)'}, {'end': 155526050L, 'name': 'FGG p.Q433P', 'start': 155526050L, 'biomarker_type': 'Splice_Site', 'referenceName': 'GRCh37', 'geneSymbol': 'FGG', 'alt': 'G', 'ref': 'T', 'chromosome': '4', 'description': 'fibrinogen gamma chain'}, {'end': 148617051L, 'name': 'ABLIM3 p.A310V', 'start': 148617051L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ABLIM3', 'alt': 'T', 'ref': 'C', 'chromosome': '5', 'description': 'actin binding LIM protein family, member 3'}, {'end': 31895941L, 'name': 'CFB p.P86S', 'start': 31895941L, 'biomarker_type': 'Splice_Site', 'referenceName': 'GRCh37', 'geneSymbol': 'CFB', 'alt': 'T', 'ref': 'C', 'chromosome': '6', 'description': 'complement factor B'}, {'end': 18051501L, 'name': 'MYO15A p.A2223V', 'start': 18051501L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'MYO15A', 'alt': 'T', 'ref': 'C', 'chromosome': '17', 'description': 'myosin XVA'}, {'end': 60969271L, 'name': 'CABLES2 p.S219C', 'start': 60969271L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CABLES2', 'alt': 'C', 'ref': 'G', 'chromosome': '20', 'description': 'Cdk5 and Abl enzyme substrate 2'}, {'end': 186096984L, 'name': 'KIAA1430 p.L426F', 'start': 186096984L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'KIAA1430', 'alt': 'A', 'ref': 'G', 'chromosome': '4', 'description': ''}, {'end': 110056501L, 'name': 'FIG4 p.G216R', 'start': 110056501L, 'biomarker_type': 'Splice_Site', 'referenceName': 'GRCh37', 'geneSymbol': 'FIG4', 'alt': 'A', 'ref': 'G', 'chromosome': '6', 'description': 'FIG4 phosphoinositide 5-phosphatase'}, {'end': 67690336L, 'name': 'RLTPR p.C1275Y', 'start': 67690336L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'RLTPR', 'alt': 'A', 'ref': 'G', 'chromosome': '16', 'description': 'RGD motif, leucine rich repeats, tropomodulin domain and proline-rich containing'}, {'end': 67702418L, 'name': 'C16orf86 p.D290G', 'start': 67702418L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'C16orf86', 'alt': 'G', 'ref': 'A', 'chromosome': '16', 'description': 'chromosome 16 open reading frame 86'}]

# p115 = [{'description': '', 'start': 27100988L, 'biomarker_type': 'substitution', 'geneSymbol': 'ARID1A', 'alt': 'T', 'ref': 'C', 'chromosome': '1', 'name': 'ARID1A '}, {'description': '', 'start': 204494718L, 'biomarker_type': 'substitution', 'geneSymbol': 'MDM4', 'alt': 'G', 'ref': 'C', 'chromosome': '1', 'name': 'MDM4 '}, {'description': '', 'start': 30143341L, 'biomarker_type': 'substitution', 'geneSymbol': 'ALK', 'alt': 'A', 'ref': 'G', 'chromosome': '2', 'name': 'ALK '}, {'description': '', 'start': 48028096L, 'biomarker_type': 'substitution', 'geneSymbol': 'MSH6', 'alt': 'C', 'ref': 'G', 'chromosome': '2', 'name': 'MSH6 '}, {'description': '', 'start': 10084764L, 'biomarker_type': 'substitution', 'geneSymbol': 'FANCD2', 'alt': 'G', 'ref': 'C', 'chromosome': '3', 'name': 'FANCD2 '}, {'description': '', 'start': 142185372L, 'biomarker_type': 'substitution', 'geneSymbol': 'ATR', 'alt': 'G', 'ref': 'C', 'chromosome': '3', 'name': 'ATR '}, {'description': '', 'start': 178916923L, 'biomarker_type': 'indel', 'geneSymbol': 'PIK3CA', 'alt': 'C', 'ref': 'CCAGTAG', 'chromosome': '3', 'name': 'PIK3CA '}, {'description': '', 'start': 117647566L, 'biomarker_type': 'substitution', 'geneSymbol': 'ROS1', 'alt': 'C', 'ref': 'G', 'chromosome': '6', 'name': 'ROS1 '}, {'description': '', 'start': 87285753L, 'biomarker_type': 'substitution', 'geneSymbol': 'NTRK2', 'alt': 'G', 'ref': 'C', 'chromosome': '9', 'name': 'NTRK2 '}, {'description': '', 'start': 108106483L, 'biomarker_type': 'substitution', 'geneSymbol': 'ATM', 'alt': 'C', 'ref': 'G', 'chromosome': '11', 'name': 'ATM '}, {'description': '', 'start': 45618180L, 'biomarker_type': 'substitution', 'geneSymbol': 'FANCM', 'alt': 'G', 'ref': 'C', 'chromosome': '14', 'name': 'FANCM '}, {'description': '', 'start': 45644454L, 'biomarker_type': 'substitution', 'geneSymbol': 'FANCM', 'alt': 'C', 'ref': 'G', 'chromosome': '14', 'name': 'FANCM '}, {'description': '', 'start': 2110733L, 'biomarker_type': 'substitution', 'geneSymbol': 'TSC2', 'alt': 'G', 'ref': 'C', 'chromosome': '16', 'name': 'TSC2 '}, {'description': 'GAIN Copy Number Ratio 3.0528', 'start': 39771328L, 'biomarker_type': 'copy_number_variation', 'geneSymbol': 'IDO1', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'IDO1 '}, {'description': 'GAIN Copy Number Ratio 5.3306', 'start': 39792474L, 'biomarker_type': 'copy_number_variation', 'geneSymbol': 'IDO2', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'IDO2 '}, {'description': 'LOSS Copy Number Ratio 0.3497', 'start': 21967751L, 'biomarker_type': 'copy_number_variation', 'geneSymbol': 'CDKN2A', 'alt': '', 'ref': '', 'chromosome': '9', 'name': 'CDKN2A '}, {'description': 'GAIN Copy Number Ratio 7.5539', 'start': 38268656L, 'biomarker_type': 'copy_number_variation', 'geneSymbol': 'FGFR1', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'FGFR1 '}, {'description': 'LOSS Copy Number Ratio 0.5662', 'start': 89623195L, 'biomarker_type': 'copy_number_variation', 'geneSymbol': 'PTEN', 'alt': '', 'ref': '', 'chromosome': '10', 'name': 'PTEN '}, {'description': 'GAIN Copy Number Ratio 2.883', 'start': 69455873L, 'biomarker_type': 'copy_number_variation', 'geneSymbol': 'CCND1', 'alt': '', 'ref': '', 'chromosome': '11', 'name': 'CCND1 '}, {'description': 'GAIN Copy Number Ratio 2.0415', 'start': 128748315L, 'biomarker_type': 'copy_number_variation', 'geneSymbol': 'MYC', 'alt': '', 'ref': '', 'chromosome': '8', 'name': 'MYC '}]  # NOQA
# p204 = [{'end': 41266125L, 'name': 'CTNNB1 p.T41I', 'start': 41266125L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'CTNNB1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'catenin (cadherin-associated protein), beta 1, 88kDa'}, {'end': 142188239L, 'name': 'ATR p.I2164M', 'start': 142188239L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'ATR', 'alt': 'C', 'ref': 'T', 'chromosome': '3', 'description': 'ATR serine/threonine kinase'}, {'end': 89685310L, 'name': 'PTEN p.-70fs', 'start': 89685309L, 'biomarker_type': 'Frame_Shift_Ins', 'geneSymbol': 'PTEN', 'alt': 'AATCT', 'ref': '-', 'chromosome': '10', 'description': 'phosphatase and tensin homolog'}, {'end': 41266125L, 'name': 'CTNNB1 p.T41I', 'start': 41266125L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'CTNNB1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'catenin (cadherin-associated protein), beta 1, 88kDa'}, {'end': 1584977L, 'name': 'P2RY8 p.A159T', 'start': 1584977L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'P2RY8', 'alt': 'T', 'ref': 'C', 'chromosome': 'X', 'description': 'purinergic receptor P2Y, G-protein coupled, 8'}, {'end': 170802031L, 'name': 'TNIK p.T1028A', 'start': 170802031L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'TNIK', 'alt': 'C', 'ref': 'T', 'chromosome': '3', 'description': 'TRAF2 and NCK interacting kinase'}, {'end': 118764345L, 'name': 'CXCR5 p.S31F', 'start': 118764345L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'CXCR5', 'alt': 'T', 'ref': 'C', 'chromosome': '11', 'description': 'chemokine (C-X-C motif) receptor 5'}, {'end': 136071095L, 'name': 'ZRANB3 p.L310F', 'start': 136071095L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'ZRANB3', 'alt': 'G', 'ref': 'C', 'chromosome': '2', 'description': 'zinc finger, RAN-binding domain containing 3'}, {'end': 228566043L, 'name': 'OBSCN p.V8839A', 'start': 228566043L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'OBSCN', 'alt': 'C', 'ref': 'T', 'chromosome': '1', 'description': 'obscurin, cytoskeletal calmodulin and titin-interacting RhoGEF'}, {'end': 2097401L, 'name': 'STK35 p.L328V', 'start': 2097401L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'STK35', 'alt': 'G', 'ref': 'C', 'chromosome': '20', 'description': 'serine/threonine kinase 35'}, {'end': 58130871L, 'name': 'AGAP2 p.A387T', 'start': 58130871L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'AGAP2', 'alt': 'T', 'ref': 'C', 'chromosome': '12', 'description': 'ArfGAP with GTPase domain, ankyrin repeat and PH domain 2'}, {'end': 33690771L, 'name': 'IP6K3 p.R320H', 'start': 33690771L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'IP6K3', 'alt': 'T', 'ref': 'C', 'chromosome': '6', 'description': 'inositol hexakisphosphate kinase 3'}, {'end': 101910837L, 'name': 'NALCN p.G408A', 'start': 101910837L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'NALCN', 'alt': 'G', 'ref': 'C', 'chromosome': '13', 'description': 'sodium leak channel, non-selective'}, {'end': 57424051L, 'name': 'MYO1A p.R845K', 'start': 57424051L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'MYO1A', 'alt': 'T', 'ref': 'C', 'chromosome': '12', 'description': 'myosin IA'}, {'end': 38211533L, 'name': 'TRPC4 p.R814K', 'start': 38211533L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'TRPC4', 'alt': 'T', 'ref': 'C', 'chromosome': '13', 'description': 'transient receptor potential cation channel, subfamily C, member 4'}, {'end': 32257834L, 'name': 'SPOCD1 p.R982C', 'start': 32257834L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'SPOCD1', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'SPOC domain containing 1'}, {'end': 28097282L, 'name': 'ZSCAN16 p.E201K', 'start': 28097282L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'ZSCAN16', 'alt': 'A', 'ref': 'G', 'chromosome': '6', 'description': 'zinc finger and SCAN domain containing 16'}, {'end': 84628811L, 'name': 'SEMA3D p.R760Q', 'start': 84628811L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'SEMA3D', 'alt': 'T', 'ref': 'C', 'chromosome': '7', 'description': 'sema domain, immunoglobulin domain (Ig), short basic domain, secreted, (semaphorin) 3D'}, {'end': 72400932L, 'name': 'NEGR1 p.A80V', 'start': 72400932L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'NEGR1', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'neuronal growth regulator 1'}, {'end': 35778772L, 'name': 'ARPP21 p.P521L', 'start': 35778772L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'ARPP21', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'cAMP-regulated phosphoprotein, 21kDa'}, {'end': 21016738L, 'name': 'KIF17 p.R442S', 'start': 21016738L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'KIF17', 'alt': 'T', 'ref': 'G', 'chromosome': '1', 'description': 'kinesin family member 17'}, {'end': 51929267L, 'name': 'IQCF1 p.R86Q', 'start': 51929267L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'IQCF1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'IQ motif containing F1'}, {'end': 95188799L, 'name': 'CDH17 p.E132K', 'start': 95188799L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'CDH17', 'alt': 'T', 'ref': 'C', 'chromosome': '8', 'description': 'cadherin 17, LI cadherin (liver-intestine)'}, {'end': 96117712L, 'name': 'CCDC82 p.L67P', 'start': 96117712L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'CCDC82', 'alt': 'G', 'ref': 'A', 'chromosome': '11', 'description': 'coiled-coil domain containing 82'}, {'end': 105967614L, 'name': 'AASDHPPT p.R304*', 'start': 105967614L, 'biomarker_type': 'Nonsense_Mutation', 'geneSymbol': 'AASDHPPT', 'alt': 'T', 'ref': 'C', 'chromosome': '11', 'description': 'aminoadipate-semialdehyde dehydrogenase-phosphopantetheinyl transferase'}, {'end': 31538698L, 'name': 'CLDN17 p.A80S', 'start': 31538698L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'CLDN17', 'alt': 'A', 'ref': 'C', 'chromosome': '21', 'description': 'claudin 17'}, {'end': 38483424L, 'name': 'UTP11L p.M70I', 'start': 38483424L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'UTP11L', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'UTP11-like, U3 small nucleolar ribonucleoprotein (yeast)'}, {'end': 155526050L, 'name': 'FGG p.Q433P', 'start': 155526050L, 'biomarker_type': 'Splice_Site', 'geneSymbol': 'FGG', 'alt': 'G', 'ref': 'T', 'chromosome': '4', 'description': 'fibrinogen gamma chain'}, {'end': 148617051L, 'name': 'ABLIM3 p.A310V', 'start': 148617051L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'ABLIM3', 'alt': 'T', 'ref': 'C', 'chromosome': '5', 'description': 'actin binding LIM protein family, member 3'}, {'end': 31895941L, 'name': 'CFB p.P86S', 'start': 31895941L, 'biomarker_type': 'Splice_Site', 'geneSymbol': 'CFB', 'alt': 'T', 'ref': 'C', 'chromosome': '6', 'description': 'complement factor B'}, {'end': 18051501L, 'name': 'MYO15A p.A2223V', 'start': 18051501L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'MYO15A', 'alt': 'T', 'ref': 'C', 'chromosome': '17', 'description': 'myosin XVA'}, {'end': 60969271L, 'name': 'CABLES2 p.S219C', 'start': 60969271L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'CABLES2', 'alt': 'C', 'ref': 'G', 'chromosome': '20', 'description': 'Cdk5 and Abl enzyme substrate 2'}, {'end': 186096984L, 'name': 'KIAA1430 p.L426F', 'start': 186096984L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'KIAA1430', 'alt': 'A', 'ref': 'G', 'chromosome': '4', 'description': ''}, {'end': 110056501L, 'name': 'FIG4 p.G216R', 'start': 110056501L, 'biomarker_type': 'Splice_Site', 'geneSymbol': 'FIG4', 'alt': 'A', 'ref': 'G', 'chromosome': '6', 'description': 'FIG4 phosphoinositide 5-phosphatase'}, {'end': 67690336L, 'name': 'RLTPR p.C1275Y', 'start': 67690336L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'RLTPR', 'alt': 'A', 'ref': 'G', 'chromosome': '16', 'description': 'RGD motif, leucine rich repeats, tropomodulin domain and proline-rich containing'}, {'end': 67702418L, 'name': 'C16orf86 p.D290G', 'start': 67702418L, 'biomarker_type': 'Missense_Mutation', 'geneSymbol': 'C16orf86', 'alt': 'G', 'ref': 'A', 'chromosome': '16', 'description': 'chromosome 16 open reading frame 86'}]

p204 = [{'end': 41266125L, 'name': 'CTNNB1 p.T41I', 'start': 41266125L, 'biomarker_type': 'Missense_Mutation', 'referenceName': '3', 'geneSymbol': 'CTNNB1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'catenin (cadherin-associated protein), beta 1, 88kDa'}, {'end': 142188239L, 'name': 'ATR p.I2164M', 'start': 142188239L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ATR', 'alt': 'C', 'ref': 'T', 'chromosome': '3', 'description': 'ATR serine/threonine kinase'}, {'end': 89685310L, 'name': 'PTEN p.-70fs', 'start': 89685309L, 'biomarker_type': 'Frame_Shift_Ins', 'referenceName': 'GRCh37', 'geneSymbol': 'PTEN', 'alt': 'AATCT', 'ref': '-', 'chromosome': '10', 'description': 'phosphatase and tensin homolog'}, {'end': 41266125L, 'name': 'CTNNB1 p.T41I', 'start': 41266125L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CTNNB1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'catenin (cadherin-associated protein), beta 1, 88kDa'}, {'end': 1584977L, 'name': 'P2RY8 p.A159T', 'start': 1584977L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'P2RY8', 'alt': 'T', 'ref': 'C', 'chromosome': 'X', 'description': 'purinergic receptor P2Y, G-protein coupled, 8'}, {'end': 170802031L, 'name': 'TNIK p.T1028A', 'start': 170802031L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'TNIK', 'alt': 'C', 'ref': 'T', 'chromosome': '3', 'description': 'TRAF2 and NCK interacting kinase'}, {'end': 118764345L, 'name': 'CXCR5 p.S31F', 'start': 118764345L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CXCR5', 'alt': 'T', 'ref': 'C', 'chromosome': '11', 'description': 'chemokine (C-X-C motif) receptor 5'}, {'end': 136071095L, 'name': 'ZRANB3 p.L310F', 'start': 136071095L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ZRANB3', 'alt': 'G', 'ref': 'C', 'chromosome': '2', 'description': 'zinc finger, RAN-binding domain containing 3'}, {'end': 228566043L, 'name': 'OBSCN p.V8839A', 'start': 228566043L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'OBSCN', 'alt': 'C', 'ref': 'T', 'chromosome': '1', 'description': 'obscurin, cytoskeletal calmodulin and titin-interacting RhoGEF'}, {'end': 2097401L, 'name': 'STK35 p.L328V', 'start': 2097401L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'STK35', 'alt': 'G', 'ref': 'C', 'chromosome': '20', 'description': 'serine/threonine kinase 35'}, {'end': 58130871L, 'name': 'AGAP2 p.A387T', 'start': 58130871L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'AGAP2', 'alt': 'T', 'ref': 'C', 'chromosome': '12', 'description': 'ArfGAP with GTPase domain, ankyrin repeat and PH domain 2'}, {'end': 33690771L, 'name': 'IP6K3 p.R320H', 'start': 33690771L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'IP6K3', 'alt': 'T', 'ref': 'C', 'chromosome': '6', 'description': 'inositol hexakisphosphate kinase 3'}, {'end': 101910837L, 'name': 'NALCN p.G408A', 'start': 101910837L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'NALCN', 'alt': 'G', 'ref': 'C', 'chromosome': '13', 'description': 'sodium leak channel, non-selective'}, {'end': 57424051L, 'name': 'MYO1A p.R845K', 'start': 57424051L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'MYO1A', 'alt': 'T', 'ref': 'C', 'chromosome': '12', 'description': 'myosin IA'}, {'end': 38211533L, 'name': 'TRPC4 p.R814K', 'start': 38211533L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'TRPC4', 'alt': 'T', 'ref': 'C', 'chromosome': '13', 'description': 'transient receptor potential cation channel, subfamily C, member 4'}, {'end': 32257834L, 'name': 'SPOCD1 p.R982C', 'start': 32257834L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'SPOCD1', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'SPOC domain containing 1'}, {'end': 28097282L, 'name': 'ZSCAN16 p.E201K', 'start': 28097282L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ZSCAN16', 'alt': 'A', 'ref': 'G', 'chromosome': '6', 'description': 'zinc finger and SCAN domain containing 16'}, {'end': 84628811L, 'name': 'SEMA3D p.R760Q', 'start': 84628811L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'SEMA3D', 'alt': 'T', 'ref': 'C', 'chromosome': '7', 'description': 'sema domain, immunoglobulin domain (Ig), short basic domain, secreted, (semaphorin) 3D'}, {'end': 72400932L, 'name': 'NEGR1 p.A80V', 'start': 72400932L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'NEGR1', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'neuronal growth regulator 1'}, {'end': 35778772L, 'name': 'ARPP21 p.P521L', 'start': 35778772L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ARPP21', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'cAMP-regulated phosphoprotein, 21kDa'}, {'end': 21016738L, 'name': 'KIF17 p.R442S', 'start': 21016738L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'KIF17', 'alt': 'T', 'ref': 'G', 'chromosome': '1', 'description': 'kinesin family member 17'}, {'end': 51929267L, 'name': 'IQCF1 p.R86Q', 'start': 51929267L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'IQCF1', 'alt': 'T', 'ref': 'C', 'chromosome': '3', 'description': 'IQ motif containing F1'}, {'end': 95188799L, 'name': 'CDH17 p.E132K', 'start': 95188799L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CDH17', 'alt': 'T', 'ref': 'C', 'chromosome': '8', 'description': 'cadherin 17, LI cadherin (liver-intestine)'}, {'end': 96117712L, 'name': 'CCDC82 p.L67P', 'start': 96117712L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CCDC82', 'alt': 'G', 'ref': 'A', 'chromosome': '11', 'description': 'coiled-coil domain containing 82'}, {'end': 105967614L, 'name': 'AASDHPPT p.R304*', 'start': 105967614L, 'biomarker_type': 'Nonsense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'AASDHPPT', 'alt': 'T', 'ref': 'C', 'chromosome': '11', 'description': 'aminoadipate-semialdehyde dehydrogenase-phosphopantetheinyl transferase'}, {'end': 31538698L, 'name': 'CLDN17 p.A80S', 'start': 31538698L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CLDN17', 'alt': 'A', 'ref': 'C', 'chromosome': '21', 'description': 'claudin 17'}, {'end': 38483424L, 'name': 'UTP11L p.M70I', 'start': 38483424L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'UTP11L', 'alt': 'A', 'ref': 'G', 'chromosome': '1', 'description': 'UTP11-like, U3 small nucleolar ribonucleoprotein (yeast)'}, {'end': 155526050L, 'name': 'FGG p.Q433P', 'start': 155526050L, 'biomarker_type': 'Splice_Site', 'referenceName': 'GRCh37', 'geneSymbol': 'FGG', 'alt': 'G', 'ref': 'T', 'chromosome': '4', 'description': 'fibrinogen gamma chain'}, {'end': 148617051L, 'name': 'ABLIM3 p.A310V', 'start': 148617051L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'ABLIM3', 'alt': 'T', 'ref': 'C', 'chromosome': '5', 'description': 'actin binding LIM protein family, member 3'}, {'end': 31895941L, 'name': 'CFB p.P86S', 'start': 31895941L, 'biomarker_type': 'Splice_Site', 'referenceName': 'GRCh37', 'geneSymbol': 'CFB', 'alt': 'T', 'ref': 'C', 'chromosome': '6', 'description': 'complement factor B'}, {'end': 18051501L, 'name': 'MYO15A p.A2223V', 'start': 18051501L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'MYO15A', 'alt': 'T', 'ref': 'C', 'chromosome': '17', 'description': 'myosin XVA'}, {'end': 60969271L, 'name': 'CABLES2 p.S219C', 'start': 60969271L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'CABLES2', 'alt': 'C', 'ref': 'G', 'chromosome': '20', 'description': 'Cdk5 and Abl enzyme substrate 2'}, {'end': 186096984L, 'name': 'KIAA1430 p.L426F', 'start': 186096984L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'KIAA1430', 'alt': 'A', 'ref': 'G', 'chromosome': '4', 'description': ''}, {'end': 110056501L, 'name': 'FIG4 p.G216R', 'start': 110056501L, 'biomarker_type': 'Splice_Site', 'referenceName': 'GRCh37', 'geneSymbol': 'FIG4', 'alt': 'A', 'ref': 'G', 'chromosome': '6', 'description': 'FIG4 phosphoinositide 5-phosphatase'}, {'end': 67690336L, 'name': 'RLTPR p.C1275Y', 'start': 67690336L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'RLTPR', 'alt': 'A', 'ref': 'G', 'chromosome': '16', 'description': 'RGD motif, leucine rich repeats, tropomodulin domain and proline-rich containing'}, {'end': 67702418L, 'name': 'C16orf86 p.D290G', 'start': 67702418L, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'C16orf86', 'alt': 'G', 'ref': 'A', 'chromosome': '16', 'description': 'chromosome 16 open reading frame 86'}]

test = [{"start": 41266125, "end": 41266125, "name": "CTNNB1 p.T41I", "referenceName": "3", "geneSymbol": "CTNNB1", "alt": "T", "description": "catenin (cadherin-associated protein), beta 1, 88kDa", "ref": "C", "chromosome": "3", "biomarker_type": "Missense_Mutation"}]
test = [{'end': 228566043, 'name': 'OBSCN p.V8839A', 'start': 228566043, 'biomarker_type': 'Missense_Mutation', 'referenceName': 'GRCh37', 'geneSymbol': 'OBSCN', 'alt': 'C', 'ref': 'T', 'chromosome': '1', 'description': 'obscurin, cytoskeletal calmodulin and titin-interacting RhoGEF'}]

dataset_features = test

dataset_name = 'test'

In [24]:
json.dumps(test)

'[{"start": 228566043, "end": 228566043, "name": "OBSCN p.V8839A", "referenceName": "GRCh37", "geneSymbol": "OBSCN", "alt": "C", "description": "obscurin, cytoskeletal calmodulin and titin-interacting RhoGEF", "ref": "T", "chromosome": "1", "biomarker_type": "Missense_Mutation"}]'

In [21]:
# run the queries and create data frames

queries = []
data_frames = {}
t = datetime.datetime.now()     

rsp = requests.post(g2p_url, json=dataset_features, timeout=240)

location_queries = rsp.json()    

evidence_ids = []

for location_query in location_queries:
    allele = location_query['allele']
    name = location_query['name']
    biomarker_type = location_query['biomarker_type']
    if allele not in data_frames:
        data_frames[allele] = {}
    data_frames[allele][name] = to_df(location_query['hits'],location_query['feature'])
    # now that we have the associations in a dataframe, lets recast the hits to a hit count
    evidence_ids.extend([hit['evidence.id'] for hit in location_query['hits']])
    location_query['hits'] = len(location_query['hits'])
    queries.append(location_query)

print 'total of {} queries, {} total hits, {} unique hits returned in {}'.format(
    len(queries),
    len(evidence_ids),
    len(list(set(evidence_ids))),
    datetime.datetime.now() - t)

total of 8 queries, 1041 total hits, 1041 unique hits returned in 0:00:12.859547


In [22]:
# summarize the queries by allele

queries_summary = {}

for q in queries:
    if q['allele'] not in queries_summary:
        queries_summary[q['allele']] = { 'allele': q['allele'], 'biomarker_type': q['biomarker_type'] }
    queries_summary[q['allele']]['hits-{}'.format(q['name'])] = q['hits']    

df = pd.DataFrame.from_records([queries_summary[k] for k in queries_summary.keys()]).fillna(0)
queries_summary_df_widget = qgrid.show_grid(df, grid_options={'editable':False})
display(queries_summary_df_widget)

QgridWidget(grid_options={'defaultColumnWidth': 150, 'highlightSelectedRow': True, 'enableTextSelectionOnCells': True, 'editable': False, 'explicitInitialization': True, 'minVisibleRows': 8, 'enableColumnReorder': False, 'maxVisibleRows': 15, 'sortable': True, 'fullWidthRows': True, 'rowHeight': 28, 'filterable': True, 'autoEdit': False, 'syncColumnCellResize': True, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

In [23]:
# create an 'interactive' master detail

from IPython.display import clear_output
from IPython.display import IFrame  
from IPython.core.display import HTML

queries_df = pd.DataFrame.from_records(queries)
queries_df_widget = qgrid.show_grid(queries_df, grid_options={'editable':False})
qgrid_widget = None
top3_widget = None
allele = None 
name = None

def top(df, fields = ['publications', 'evidence_label', 'phenotype', 'drugs' ], size=3):
    counters = []
    for f in fields:
        counter = Counter()
        for value in df[f]:
            if isinstance(value, (list,tuple,)):
                for v in value:
                    counter[v] += 1
            else:
                counter[value] += 1
        for c in counter.most_common(size):
            counters.append({'field': f, 'value': c[0], 'count': c[1]})
    if len(counters) == 0:
        counters.append({'field': fields, 'value': None, 'count': None})
    return counters    
    

def on_queries_df_row_selected(change):
    global allele
    global name
    global qgrid_widget
    global top3_widget    
    
    selected_row = queries_df_widget.get_selected_df().to_dict('records')
    allele = selected_row[0]['allele']
    name = selected_row[0]['name']
    query_string = selected_row[0]['query_string']
        
    top3_widget = None
    
    try:    
        df = data_frames[allele][name]
        qgrid_widget = qgrid.show_grid(df)
        top3_widget = qgrid.show_grid(pd.DataFrame(top(df)))
        
    except Exception as e:
        # df could be empty
        pass  
    
    # show publication
    def on_row_selected(change):
        global qgrid_widget
        try:
            selected_row = qgrid_widget.get_selected_df().to_dict('records')
            publications = selected_row[0]['publications']
            matches = selected_row[0]['matches']            
            clear_output(wait=True)
            display(queries_df_widget,
                    HTML('<p>{} {}</p><p>{}</p>'.format(allele, name, query_string)),
                    qgrid_widget,
                    HTML('<p>matches:{}</p><p>{}</p>'.format(matches, publications)),
                    IFrame(publications[0], width=900, height=900))    
        except Exception as e:          
            print e

    qgrid_widget.observe(on_row_selected, names=['_selected_rows'])        
    clear_output(wait=True)
    try:
        if top3_widget:
            display(HTML('<p>query summary:</p>'),
                    queries_df_widget,
                    HTML('<p>top3:</p>'),
                    top3_widget,
                    HTML('<p>{} {}</p><p>{}</p>'.format(allele, name, query_string)),
                    qgrid_widget)    
        else:
            display(queries_df_widget, HTML('<p>No results</p>'))
    except Exception as e:          
        print e        
    
queries_df_widget.observe(on_queries_df_row_selected, names=['_selected_rows'])   # ,  

display(HTML('<p>query summary:</p>'),queries_df_widget)

QgridWidget(grid_options={'defaultColumnWidth': 150, 'highlightSelectedRow': True, 'enableTextSelectionOnCells': True, 'editable': False, 'explicitInitialization': True, 'minVisibleRows': 8, 'enableColumnReorder': False, 'maxVisibleRows': 15, 'sortable': True, 'fullWidthRows': True, 'rowHeight': 28, 'filterable': True, 'autoEdit': False, 'syncColumnCellResize': True, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

In [78]:

def collect_md(s, md=None):
    s = s.decode('UTF-8', 'ignore')
    if md:
        md = u'{}\n{}'.format(md, s)
    else:
        md = u'{}'.format(s)
    return md

# report parameters
query_names = ['alleles', '~location', '~range', 'protein_effects', 'protein_domain', '~biomarker_type',  'genes' , 'pathways']
evidence_levels = ['A', 'B', 'C', 'D']
# maximum number of matching evidence items
limit = 2

In [72]:

 
md = collect_md('# {}'.format(dataset_name))

md = collect_md('\n## Drug Centric\n_____', md)

for name in query_names:
    df = pd.DataFrame()
    for allele in data_frames.keys():
        if name in data_frames[allele] and len(data_frames[allele][name]):
            df = df.append(data_frames[allele][name])
    if len(df) == 0:
        continue
    md = collect_md('\n### {}\n'.format(name), md)
    for evidence_level in evidence_levels:
        evidence_level_df = df[df.evidence_label == evidence_level]
        if len(evidence_level_df):
            drug = top(evidence_level_df, fields=['drugs'], size=1)[0]['value']
            md = collect_md('  * {} {}'.format(evidence_level, drug), md)
            c = 0
            for evidence in evidence_level_df.to_records():
                if drug in evidence.drugs:
                    md = collect_md('    * {} {} {}'.format(evidence.source, evidence.phenotype, evidence.drugs), md)
                    md = collect_md('      * {}'.format(evidence.description), md)
                    md = collect_md('      * {}'.format(evidence.matches), md)                    
                    md = collect_md('      * {}'.format(evidence.publications), md)                                        
                    c += 1
                    if c == limit:
                        break
                        
md = collect_md('\n## Phenotype Centric\n_____', md)

for name in query_names:
    df = pd.DataFrame()
    for allele in data_frames.keys():
        if name in data_frames[allele] and len(data_frames[allele][name]):
            df = df.append(data_frames[allele][name])
    if len(df) == 0:
        continue
    md = collect_md('\n### {}\n'.format(name), md)
    for evidence_level in evidence_levels:
        evidence_level_df = df[df.evidence_label == evidence_level]
        if len(evidence_level_df):
            phenotype = top(evidence_level_df, fields=['phenotype'], size=1)[0]['value']
            md = collect_md('  * {} {}'.format(evidence_level, phenotype), md)
            c = 0
            for evidence in evidence_level_df.to_records():
                if phenotype in evidence.phenotype:
                    md = collect_md('    * {} {} {}'.format(evidence.source, evidence.phenotype, evidence.drugs), md)
                    md = collect_md('      * {}'.format(evidence.description), md)
                    md = collect_md('      * {}'.format(evidence.matches), md)                    
                    md = collect_md('      * {}'.format(evidence.publications), md)                                        
                    c += 1
                    if c == limit:
                        break

                        
                        
md = collect_md('\n## Publication Centric\n_____', md)

for name in query_names:
    df = pd.DataFrame()
    for allele in data_frames.keys():
        if name in data_frames[allele] and len(data_frames[allele][name]):
            df = df.append(data_frames[allele][name])
    if len(df) == 0:
        continue
    md = collect_md('\n### {}\n'.format(name), md)
    for evidence_level in evidence_levels:
        evidence_level_df = df[df.evidence_label == evidence_level]
        if len(evidence_level_df):
            publication = top(evidence_level_df, fields=['publications'], size=1)[0]['value']
            md = collect_md('  * {} {}'.format(evidence_level, publication), md)
            c = 0
            for evidence in evidence_level_df.to_records():
                if publication in evidence.publications:
                    md = collect_md('    * {} {} {}'.format(evidence.source, evidence.phenotype, evidence.drugs), md)
                    md = collect_md('      * {}'.format(evidence.description), md)
                    md = collect_md('      * {}'.format(evidence.matches), md)                   
                    md = collect_md('      * {}'.format(evidence.publications), md)                    
                    c += 1
                    if c == limit:
                        break

                        
                        
# print md    
Markdown(md)



# p204

## Drug Centric
_____

### ~location

  * B mAb
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C SIROLIMUS
    * cgi cancer ['SIROLIMUS']
      * PTEN Sirolimus (MTOR inhibitor) Responsive
      * [u'PTEN']
      * [u'https://www.google.com/#q=ASCO 2013 (abstr 2532)']

### ~range

  * B mAb
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C EVEROLIMUS
    * jax thyroid medullary carcinoma ['VANDETANIB', 'EVEROLIMUS']
      * In a clinical case study, addition of Afinitor (everolimus)  to Caprelsa (vandetanib) treatment resulted in significant tumor reduction in a medullary thyroid carcinoma patient harboring ATM L804fs*4, ATM S978fs*12, and RET M918T, that achieved prolonged stable disease on Caprelsa (vandetanib) treatment alone (PMID: 27683183).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27683183']
    * jax thyroid carcinoma ['DABRAFENIB', 'EVEROLIMUS', 'TRAMETINIB']
      * In a clinical case study, a patient with anaplastic thyroid carcinoma co-harboring BRAF V600E and PIK3CA H1047R demonstrated tumor regression when treated with the triple combination, Tafinlar (dabrafenib), Mekinist (trametinib), and Afinitor (everolimus) (PMID: 27797976).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27797976']
  * D TRAMETINIB
    * jax melanoma ['TRAMETINIB']
      * In a preclinical study, Mekinist (trametinib) reduced ERK signaling and inhibited proliferation of a melanoma cell line harboring BRAF G466E and HRAS Q61K in culture (PMID: 28783719).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/28783719']
    * jax melanoma ['TRAMETINIB']
      * In a preclinical study, Mekinist (trametinib) in combination with Omipalisib (GSK2126458) resulted in improved growth inhibition in melanoma cell lines harboring BRAF V600E, NRAS Q61K, NRAS A146T and MAP2K1 P387S in culture, compared to either agent alone (PMID: 22389471).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/22389471']

### protein_effects

  * B mAb
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C Sorafenib
    * civic non-small cell lung carcinoma ['Sorafenib']
      * In one patient with S214C mutation, the use of sorafenib has led to more than 5 years of survival and near remission.
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24569458']
    * oncokb histiocytosis ['Sorafenib']
      * Compelling clinical evidence supports the biomarker as being predictive of response to a drug in this indication but neither biomarker and drug are standard of care
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24569458', u'http://www.ncbi.nlm.nih.gov/pubmed/17016424', u'http://www.ncbi.nlm.nih.gov/pubmed/26566875']
  * D Sorafenib
    * civic non-small cell lung carcinoma ['Sorafenib']
      * Sorafenib can inhibit ARF-mediated MEK phosphorylation and soft agar colony formation of AALE cells expressing ARAF S214C in vitro.
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24569458']

### ~biomarker_type

  * B GEFITINIB
    * civic glioblastoma multiforme ['GEFITINIB', 'Erlotinib']
      * Expression of EGFRvIII and PTEN was evaluated in 26 patients with glioblastomas treated with erlotinib or gefitinib and no EGFR or Her2/neu kinase domain mutations. 7 patients responded to treatment while 19 progressed rapidly. Coexpression of EGFRvIII and PTEN was associated with clinical response (defined as 25% tumor shrinkage). These results were confirmed using IHC in an independent cohort of 33 patients (8 responders) treated with erlotinib at another institution.
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/16282176']
  * C EVEROLIMUS
    * civic Her2-receptor positive breast cancer ['FULVESTRANT', 'EVEROLIMUS']
      * Case report of a 44-year-old patient with recurrent HER2-positive breast cancer. The primary tumor was HR positive; however, the metastatic tumor was HR negative. The patient was resistant to classical chemotherapeutic agents and anti-HER2 treatment. Thus, the combination of everolimus and fulvestrant was administered. The patient experienced stable disease and PFS of 10 months. Genetic testing of pleural effusion after disease progression revealed a PIK3CA H1047R gene mutation but was positive for PTEN expression (IHC).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27445490']
    * jax renal cell carcinoma ['EVEROLIMUS']
      * In a retrospective analysis from a Phase II clinical trial, patients with metastatic renal cell carcinoma treated with Afinitor (everolimus) had no difference in progression free survival when stratified by high expression (n=17) and low expression (n=21) of Pten (PMID: 26951309).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26951309']
  * D VEMURAFENIB
    * jax melanoma ['VEMURAFENIB']
      * In a preclinical study, expression of AKT1 E17K in a wild-type PTEN-expressing BRAF V600-mutant melanoma cell line conferred resistance to Zelboraf (vemurafenib) in culture (PMID: 24265152).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24265152']
    * jax melanoma ['VEMURAFENIB']
      * In a preclinical study, expression of AKT1 Q79K in a wild-type PTEN-expressing BRAF V600-mutant melanoma cell line conferred resistance to Zelboraf (vemurafenib) in culture (PMID: 24265152).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24265152']

### genes

  * B EVEROLIMUS
    * civic endometrial carcinoma ['EVEROLIMUS']
      * Tumors from 35 patients (12 with partial response or stable disease) with advanced endometrial carcinoma were assessed for PTEN mutational status, revealing 12 tumors with non-identical PTEN gene mutations. No mutations found in PTEN were effective in predicting tumor response to everolimus treatment. The clinical trial associated with this study is NCT00870337.
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/23238879']
    * civic Her2-receptor positive breast cancer ['EVEROLIMUS']
      * Archival tumor samples from 549 patients from the BOLERO-1 and -3 trials were analyzed for alterations in the PI3K pathway using next-generation sequencing, immunohistochemistry (IHC) and Sanger sequencing. In the BOLERO trials, patients with HER2-overexpressing advanced breast cancer were treated with trastuzumab, chemotherapy and randomized to receive either everolimus or placebo. In the pooled analysis of both trials, patients with PTEN loss (as defined by IHC staining intensity and proportion of positive tumor cells) had a longer progression-free survival with everolimus compared to placebo (HR 0.54 (95%CI 0.31 to 0.96, P=0.04) whereas patients with PTEN normal status had a hazard ration of 1.00 (95%CI 0.8 to 1.26, P=0.97).
The same was found when PI3K mutations, PTEN loss and AKT E17K mutations were combined and analyzed as hyperactive PI3K pathway. Patients with PI3K hyperactive pathway had a hazard ratio of 0.67 (95%CI 0.48 to 0.93, P=0.02) compared to HR 1.19 (95%CI 0.87 to 1.62, P=0.28) in patients with PI3K normal pathway. These trends were independently confirmed in both trials but only pooled results showed consistent statistical significance.
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C EVEROLIMUS
    * civic Her2-receptor positive breast cancer ['FULVESTRANT', 'EVEROLIMUS']
      * Case report of a 44-year-old patient with recurrent HER2-positive breast cancer. The primary tumor was HR positive; however, the metastatic tumor was HR negative. The patient was resistant to classical chemotherapeutic agents and anti-HER2 treatment. Thus, the combination of everolimus and fulvestrant was administered. The patient experienced stable disease and PFS of 10 months. Genetic testing of pleural effusion after disease progression revealed a PIK3CA H1047R gene mutation but was positive for PTEN expression (IHC).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27445490']
    * cgi prostate adenocarcinoma ['EVEROLIMUS']
      * PTEN Everolimus (MTOR inhibitor) Responsive
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/23582881']
  * D EVEROLIMUS
    * civic breast cancer ['EVEROLIMUS', 'PP242']
      * In a panel of 31 breast cancer cell lines, 11 had loss of PTEN function. Cells were tested for sensitivity to mTORC1 inhibitor everolimus and mTORC1/2 inhibitor PP242, and no significant correlation was found between PTEN status and sensitivity to mTORC inhibition.
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/21358673']
    * jax breast cancer ['EVEROLIMUS', 'SELUMETINIB']
      * In a preclinical study, breast cancer cells harboring PIK3CA wild-type and PTEN loss did not demonstrate any benefit when treated with a combination of Afinitor (everolimus) and Selumetinib (AZD6244) (PMID: 21358673).
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/21358673']

### pathways

  * B None
  * C Imatinib
    * jax sarcoma ['Imatinib']
      * In a retrospective analysis, patients with desmoid fibromatosis harboring CTNNB1 S45F demonstrated a greater progression arrest rate at 6 months (85%) compared to patients with CTNNB1 wild-type (45%) when treated with Gleevec (imatinib) (PMID: 26861905). 
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26861905']
    * jax sarcoma ['Imatinib']
      * In a retrospective analysis, patients with desmoid fibromatosis harboring CTNNB1 T41A demonstrated a greater progression arrest rate at 6 months (70%) compared to patients with CTNNB1 wild-type (45%) when treated with Gleevec (imatinib) (PMID: 26861905). 
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26861905']
  * D LGK
    * molecularmatch rectal neoplasm ['LGK']
      * CTNNB1 S37Y confers sensitivity to Lgk-974 in patients with Neoplasm of colorectum
      * []
      * [u'https://www.ncbi.nlm.nih.gov/pubmed/10347231', u'https://www.ncbi.nlm.nih.gov/pubmed/9065403']
    * molecularmatch rectal neoplasm ['LGK']
      * CTNNB1 T41A confers sensitivity to Lgk-974 in patients with Neoplasm of colorectum
      * []
      * [u'https://www.ncbi.nlm.nih.gov/pubmed/10698519', u'https://www.ncbi.nlm.nih.gov/pubmed/10487827', u'https://www.ncbi.nlm.nih.gov/pubmed/12200448']

## Phenotype Centric
_____

### ~location

  * B breast adenocarcinoma
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C prostate adenocarcinoma
    * cgi prostate adenocarcinoma ['EVEROLIMUS']
      * PTEN Everolimus (MTOR inhibitor) Responsive
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/23582881']

### ~range

  * B breast adenocarcinoma
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C melanoma
    * jax melanoma ['VEMURAFENIB']
      * In a clinical case study, a patient harboring BRAF V600E demonstrated an initial response to treatment with Zelboraf (vemurafenib), but progressed following emergence of a MAP2K1 C121S mutation (PMID: 21383288).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/21383288']
    * jax melanoma ['VEMURAFENIB']
      * In a clinical study, a melanoma patient harboring a BRAF V600 mutation developed resistant mutations, MAP2K1 V60E, NRAS T58I, and NRAS Q61R, after 18 weeks of treatment with Zelboraf (vemurafenib) (PMID: 24265153).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24265153']
  * D colorectal cancer
    * jax colorectal cancer ['PALBOCICLIB', 'TRAMETINIB']
      * In a preclinical study, Mekinist (trametinib) and Ibrance (palbociclib)  combination treatment did not result in enhanced antitumor activity compared to Ibrance (palbociclib) alone in PDX models of KRAS wild-type colorectal cancer harboring BRAF V600E (PMID: 26369631).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26369631']
    * jax colorectal cancer ['VEMURAFENIB']
      * In a preclinical study, colorectal cancer cells harboring BRAF V600E that acquired EGFR amplification and subsequent resistance to Selumetinib (AZD6244) and Zelboraf (vemurafenib) combination treatment were resistant to Zelboraf (vemurafenib) in culture (PMID: 27312529).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27312529']

### protein_effects

  * B breast adenocarcinoma
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C prostate adenocarcinoma
    * cgi prostate adenocarcinoma ['EVEROLIMUS']
      * PTEN Everolimus (MTOR inhibitor) Responsive
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/23582881']
    * cgi prostate adenocarcinoma ['EVEROLIMUS']
      * PTEN Everolimus (MTOR inhibitor) Responsive
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/23582881']
  * D non-small cell lung carcinoma
    * civic non-small cell lung carcinoma ['Sorafenib']
      * Sorafenib can inhibit ARF-mediated MEK phosphorylation and soft agar colony formation of AALE cells expressing ARAF S214C in vitro.
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24569458']
    * civic non-small cell lung carcinoma ['TRAMETINIB']
      * Trametinib can inhibit ARF-mediated ERK phosphorylation and soft agar colony formation of AALE cells expressing ARAF S214C in vitro.
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24569458']

### ~biomarker_type

  * B glioblastoma multiforme
    * civic glioblastoma multiforme ['GEFITINIB', 'Erlotinib']
      * Expression of EGFRvIII and PTEN was evaluated in 26 patients with glioblastomas treated with erlotinib or gefitinib and no EGFR or Her2/neu kinase domain mutations. 7 patients responded to treatment while 19 progressed rapidly. Coexpression of EGFRvIII and PTEN was associated with clinical response (defined as 25% tumor shrinkage). These results were confirmed using IHC in an independent cohort of 33 patients (8 responders) treated with erlotinib at another institution.
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/16282176']
  * C renal cell carcinoma
    * jax renal cell carcinoma ['EVEROLIMUS']
      * In a retrospective analysis from a Phase II clinical trial, patients with metastatic renal cell carcinoma treated with Afinitor (everolimus) had no difference in progression free survival when stratified by high expression (n=17) and low expression (n=21) of Pten (PMID: 26951309).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26951309']
    * jax renal cell carcinoma ['EVEROLIMUS']
      * In a retrospective analysis from a Phase II clinical trial, patients with metastatic renal cell carcinoma treated with Afinitor (everolimus) had no difference in progression free survival when stratified by the presence (n=21) or absence (n=17) of PIK3CA activating mutations (PMID: 26951309).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26951309']
  * D melanoma
    * jax melanoma ['VEMURAFENIB']
      * In a preclinical study, expression of AKT1 E17K in a wild-type PTEN-expressing BRAF V600-mutant melanoma cell line conferred resistance to Zelboraf (vemurafenib) in culture (PMID: 24265152).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24265152']
    * jax melanoma ['VEMURAFENIB']
      * In a preclinical study, expression of AKT1 Q79K in a wild-type PTEN-expressing BRAF V600-mutant melanoma cell line conferred resistance to Zelboraf (vemurafenib) in culture (PMID: 24265152).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24265152']

### genes

  * B prostate cancer
    * civic prostate cancer ['EVEROLIMUS']
      * In a phase II trial on castration-resistant prostate cancer patients, FISH detection of PTEN deletion was associated with increased progression free survival (p=0.07) and with improved PSA response (p=0.02) in response to everolimus, an mTOR pathway inhibitor.
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/23582881']
  * C cancer
    * civic Her2-receptor positive breast cancer ['FULVESTRANT', 'EVEROLIMUS']
      * Case report of a 44-year-old patient with recurrent HER2-positive breast cancer. The primary tumor was HR positive; however, the metastatic tumor was HR negative. The patient was resistant to classical chemotherapeutic agents and anti-HER2 treatment. Thus, the combination of everolimus and fulvestrant was administered. The patient experienced stable disease and PFS of 10 months. Genetic testing of pleural effusion after disease progression revealed a PIK3CA H1047R gene mutation but was positive for PTEN expression (IHC).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27445490']
    * jax cancer ['PEMETREXED', 'Sorafenib']
      * In a Phase I clinical trial, low Pten expression or lack of Pten expression was associated with better response to treatment with the combination of Alimta (pemetrexed) and Nexavar (sorafenib) in patients with advanced solid tumors (PMID: 27213589).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27213589']
  * D breast cancer
    * civic breast cancer ['EVEROLIMUS', 'PP242']
      * In a panel of 31 breast cancer cell lines, 11 had loss of PTEN function. Cells were tested for sensitivity to mTORC1 inhibitor everolimus and mTORC1/2 inhibitor PP242, and no significant correlation was found between PTEN status and sensitivity to mTORC inhibition.
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/21358673']
    * jax breast cancer ['EVEROLIMUS', 'SELUMETINIB']
      * In a preclinical study, breast cancer cells harboring PIK3CA wild-type and PTEN loss did not demonstrate any benefit when treated with a combination of Afinitor (everolimus) and Selumetinib (AZD6244) (PMID: 21358673).
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/21358673']

### pathways

  * B Desmoid Fibromatosis
    * civic Desmoid Fibromatosis []
      * In this study a collection of 254 sporadic desmoid tumors were analysed for presence of mutations in CTNNB1. A mutation in exon 3 of the CTNNB1 gene was identified in 223 (88%) of the 254 cases. The CTNNB1 mutation spectra concerned mainly three point mutations in two codons (41 and 45): T41A was identified in 132 cases (59%); S45F was identified in 49 samples (22%); and S45P was identified in 27 samples (12%). None of the 175 lesions mimicking desmoid tumors analyzed harbored any CTNNB1 mutations.
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/22766794']
    * civic Desmoid Fibromatosis []
      * Desmoid fibromatosis is a rare, nonmetastatic neoplasm marked by local invasiveness and relentless recurrence. Beta-Catenin deregulation has been commonly identified in sporadic desmoids. CTNNB1 mutations were observed in 117 of 138 (85%) of desmoids. Three discrete mutations in two codons of CTNNB1 exon 3 were identified: T41A (59%), S45F (33%), and S45P (8%). Five-year recurrence-free survival was significantly poorer in S45F-mutated desmoids (23%, P < 0.0001) versus either T41A (57%) or nonmutated tumors (65%). Because of their low frequency, patients with S45P mutations were excluded from these analyses but showed a recurrence course similar to T41A or wild type patients.
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/18832571']
  * C sarcoma
    * jax sarcoma ['Imatinib']
      * In a retrospective analysis, patients with desmoid fibromatosis harboring CTNNB1 S45F demonstrated a greater progression arrest rate at 6 months (85%) compared to patients with CTNNB1 wild-type (45%) when treated with Gleevec (imatinib) (PMID: 26861905). 
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26861905']
    * jax sarcoma ['Imatinib']
      * In a retrospective analysis, patients with desmoid fibromatosis harboring CTNNB1 T41A demonstrated a greater progression arrest rate at 6 months (70%) compared to patients with CTNNB1 wild-type (45%) when treated with Gleevec (imatinib) (PMID: 26861905). 
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26861905']
  * D rectal neoplasm
    * molecularmatch rectal neoplasm ['LGK']
      * CTNNB1 S37Y confers sensitivity to Lgk-974 in patients with Neoplasm of colorectum
      * []
      * [u'https://www.ncbi.nlm.nih.gov/pubmed/10347231', u'https://www.ncbi.nlm.nih.gov/pubmed/9065403']
    * molecularmatch rectal neoplasm ['LGK']
      * CTNNB1 T41A confers sensitivity to Lgk-974 in patients with Neoplasm of colorectum
      * []
      * [u'https://www.ncbi.nlm.nih.gov/pubmed/10698519', u'https://www.ncbi.nlm.nih.gov/pubmed/10487827', u'https://www.ncbi.nlm.nih.gov/pubmed/12200448']

## Publication Centric
_____

### ~location

  * B http://www.ncbi.nlm.nih.gov/pubmed/27091708
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C http://www.ncbi.nlm.nih.gov/pubmed/26861905
    * jax sarcoma ['Imatinib']
      * In a retrospective analysis, patients with desmoid fibromatosis harboring CTNNB1 T41A demonstrated a greater progression arrest rate at 6 months (70%) compared to patients with CTNNB1 wild-type (45%) when treated with Gleevec (imatinib) (PMID: 26861905). 
      * [u'CTNNB1', u'missense_variant', u'P35222', 'pathway']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26861905']

### ~range

  * B http://www.ncbi.nlm.nih.gov/pubmed/27091708
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C http://www.ncbi.nlm.nih.gov/pubmed/28514312
    * jax colon cancer ['VEMURAFENIB', 'PANITUMUMAB']
      * In a clinical study, a colon cancer patient harboring BRAF V600E and TP53 Q192K demonstrated a partial response when treated with a combination of Zelboraf (vemurafenib) and Vectibix (panitumumab) (PMID: 28514312). 
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/28514312']
    * jax melanoma ['Imatinib', 'CARBOPLATIN', 'PACLITAXEL']
      * In a clinical study, a melanoma patient harboring a PDGFRA mutation, CDKN2A mutation, MET exon 14 splice site mutation, and SMAD4 Q249H demonstrated a 21 month overall survival when treated with a combination of Gleevec (imatinib), Paraplatin (carboplatin), and Taxol (paclitaxel) (PMID: 28514312).
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/28514312']
  * D http://www.ncbi.nlm.nih.gov/pubmed/27312529
    * jax colorectal cancer ['VEMURAFENIB']
      * In a preclinical study, colorectal cancer cells harboring BRAF V600E that acquired EGFR amplification and subsequent resistance to Selumetinib (AZD6244) and Zelboraf (vemurafenib) combination treatment were resistant to Zelboraf (vemurafenib) in culture (PMID: 27312529).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27312529']
    * jax colorectal cancer ['VEMURAFENIB', 'CETUXIMAB']
      * In a preclinical study, colorectal cancer cells harboring BRAF V600E that acquired KRAS amplification and subsequent resistance to Erbitux (cetuximab) and Tafinlar (dabrafenib) combination treatment were resistant to combination therapy consisting of Erbitux (cetuximab) and Zelboraf (vemurafenib) in culture (PMID: 27312529).
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27312529']

### protein_effects

  * B http://www.ncbi.nlm.nih.gov/pubmed/27091708
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C http://www.ncbi.nlm.nih.gov/pubmed/24569458
    * civic non-small cell lung carcinoma ['Sorafenib']
      * In one patient with S214C mutation, the use of sorafenib has led to more than 5 years of survival and near remission.
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24569458']
    * oncokb histiocytosis ['Sorafenib']
      * Compelling clinical evidence supports the biomarker as being predictive of response to a drug in this indication but neither biomarker and drug are standard of care
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24569458', u'http://www.ncbi.nlm.nih.gov/pubmed/17016424', u'http://www.ncbi.nlm.nih.gov/pubmed/26566875']
  * D http://www.ncbi.nlm.nih.gov/pubmed/24569458
    * civic non-small cell lung carcinoma ['Sorafenib']
      * Sorafenib can inhibit ARF-mediated MEK phosphorylation and soft agar colony formation of AALE cells expressing ARAF S214C in vitro.
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24569458']
    * civic non-small cell lung carcinoma ['TRAMETINIB']
      * Trametinib can inhibit ARF-mediated ERK phosphorylation and soft agar colony formation of AALE cells expressing ARAF S214C in vitro.
      * [u'missense_variant']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24569458']

### ~biomarker_type

  * B http://www.ncbi.nlm.nih.gov/pubmed/16282176
    * civic glioblastoma multiforme ['GEFITINIB', 'Erlotinib']
      * Expression of EGFRvIII and PTEN was evaluated in 26 patients with glioblastomas treated with erlotinib or gefitinib and no EGFR or Her2/neu kinase domain mutations. 7 patients responded to treatment while 19 progressed rapidly. Coexpression of EGFRvIII and PTEN was associated with clinical response (defined as 25% tumor shrinkage). These results were confirmed using IHC in an independent cohort of 33 patients (8 responders) treated with erlotinib at another institution.
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/16282176']
  * C http://www.ncbi.nlm.nih.gov/pubmed/26951309
    * jax renal cell carcinoma ['EVEROLIMUS']
      * In a retrospective analysis from a Phase II clinical trial, patients with metastatic renal cell carcinoma treated with Afinitor (everolimus) had no difference in progression free survival when stratified by high expression (n=17) and low expression (n=21) of Pten (PMID: 26951309).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26951309']
    * jax renal cell carcinoma ['EVEROLIMUS']
      * In a retrospective analysis from a Phase II clinical trial, patients with metastatic renal cell carcinoma treated with Afinitor (everolimus) had no difference in progression free survival when stratified by the presence (n=21) or absence (n=17) of PIK3CA activating mutations (PMID: 26951309).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26951309']
  * D http://www.ncbi.nlm.nih.gov/pubmed/24265152
    * jax melanoma ['VEMURAFENIB']
      * In a preclinical study, expression of AKT1 E17K in a wild-type PTEN-expressing BRAF V600-mutant melanoma cell line conferred resistance to Zelboraf (vemurafenib) in culture (PMID: 24265152).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24265152']
    * jax melanoma ['VEMURAFENIB']
      * In a preclinical study, expression of AKT1 Q79K in a wild-type PTEN-expressing BRAF V600-mutant melanoma cell line conferred resistance to Zelboraf (vemurafenib) in culture (PMID: 24265152).
      * [u'PTEN', u'Uncategorized']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/24265152']

### genes

  * B http://www.ncbi.nlm.nih.gov/pubmed/27091708
    * civic Her2-receptor positive breast cancer ['EVEROLIMUS']
      * Archival tumor samples from 549 patients from the BOLERO-1 and -3 trials were analyzed for alterations in the PI3K pathway using next-generation sequencing, immunohistochemistry (IHC) and Sanger sequencing. In the BOLERO trials, patients with HER2-overexpressing advanced breast cancer were treated with trastuzumab, chemotherapy and randomized to receive either everolimus or placebo. In the pooled analysis of both trials, patients with PTEN loss (as defined by IHC staining intensity and proportion of positive tumor cells) had a longer progression-free survival with everolimus compared to placebo (HR 0.54 (95%CI 0.31 to 0.96, P=0.04) whereas patients with PTEN normal status had a hazard ration of 1.00 (95%CI 0.8 to 1.26, P=0.97).
The same was found when PI3K mutations, PTEN loss and AKT E17K mutations were combined and analyzed as hyperactive PI3K pathway. Patients with PI3K hyperactive pathway had a hazard ratio of 0.67 (95%CI 0.48 to 0.93, P=0.02) compared to HR 1.19 (95%CI 0.87 to 1.62, P=0.28) in patients with PI3K normal pathway. These trends were independently confirmed in both trials but only pooled results showed consistent statistical significance.
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
    * cgi breast adenocarcinoma ['EVEROLIMUS', 'TRASTUZUMAB', 'mAb']
      * PTEN ERBB2 Everolimus + Trastuzumab + Chemotherapy (MTOR inhibitor + ERBB2 mAb inhibitor + Chemotherapy) Responsive
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/27091708']
  * C http://www.ncbi.nlm.nih.gov/pubmed/26861905
    * jax sarcoma ['Imatinib']
      * In a retrospective analysis, patients with desmoid fibromatosis harboring CTNNB1 S45F demonstrated a greater progression arrest rate at 6 months (85%) compared to patients with CTNNB1 wild-type (45%) when treated with Gleevec (imatinib) (PMID: 26861905). 
      * [u'CTNNB1', u'missense_variant', u'P35222', 'pathway']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26861905']
    * jax sarcoma ['Imatinib']
      * In a retrospective analysis, patients with desmoid fibromatosis harboring CTNNB1 activating mutations demonstrated a greater progression arrest rate at 6 months compared to patients with CTNNB1 wild-type when treated with Gleevec (imatinib) (PMID: 26861905).
      * [u'CTNNB1']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26861905']
  * D http://www.ncbi.nlm.nih.gov/pubmed/21358673
    * civic breast cancer ['EVEROLIMUS', 'PP242']
      * In a panel of 31 breast cancer cell lines, 11 had loss of PTEN function. Cells were tested for sensitivity to mTORC1 inhibitor everolimus and mTORC1/2 inhibitor PP242, and no significant correlation was found between PTEN status and sensitivity to mTORC inhibition.
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/21358673']
    * jax breast cancer ['EVEROLIMUS', 'SELUMETINIB']
      * In a preclinical study, breast cancer cells harboring PIK3CA wild-type and PTEN loss did not demonstrate any benefit when treated with a combination of Afinitor (everolimus) and Selumetinib (AZD6244) (PMID: 21358673).
      * [u'PTEN']
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/21358673']

### pathways

  * B http://www.ncbi.nlm.nih.gov/pubmed/18832571
    * civic Desmoid Fibromatosis []
      * Desmoid fibromatosis is a rare, nonmetastatic neoplasm marked by local invasiveness and relentless recurrence. Beta-Catenin deregulation has been commonly identified in sporadic desmoids. CTNNB1 mutations were observed in 117 of 138 (85%) of desmoids. Three discrete mutations in two codons of CTNNB1 exon 3 were identified: T41A (59%), S45F (33%), and S45P (8%). Five-year recurrence-free survival was significantly poorer in S45F-mutated desmoids (23%, P < 0.0001) versus either T41A (57%) or nonmutated tumors (65%). Because of their low frequency, patients with S45P mutations were excluded from these analyses but showed a recurrence course similar to T41A or wild type patients.
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/18832571']
    * civic Desmoid Fibromatosis []
      * Desmoid fibromatosis is a rare, nonmetastatic neoplasm marked by local invasiveness and relentless recurrence. Beta-Catenin deregulation has been commonly identified in sporadic desmoids.
CTNNB1 mutations were observed in 117 of 138 (85%) of desmoids. Three discrete mutations in two codons of CTNNB1 exon 3 were identified: T41A (59%), S45F (33%), and S45P (8%). In conclusion, CTNNB1 mutations are highly common in desmoid tumors.
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/18832571']
  * C http://www.ncbi.nlm.nih.gov/pubmed/26861905
    * jax sarcoma ['Imatinib']
      * In a retrospective analysis, patients with desmoid fibromatosis harboring CTNNB1 S45F demonstrated a greater progression arrest rate at 6 months (85%) compared to patients with CTNNB1 wild-type (45%) when treated with Gleevec (imatinib) (PMID: 26861905). 
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26861905']
    * jax sarcoma ['Imatinib']
      * In a retrospective analysis, patients with desmoid fibromatosis harboring CTNNB1 T41A demonstrated a greater progression arrest rate at 6 months (70%) compared to patients with CTNNB1 wild-type (45%) when treated with Gleevec (imatinib) (PMID: 26861905). 
      * []
      * [u'http://www.ncbi.nlm.nih.gov/pubmed/26861905']
  * D https://www.ncbi.nlm.nih.gov/pubmed/10347231
    * molecularmatch rectal neoplasm ['LGK']
      * CTNNB1 S37Y confers sensitivity to Lgk-974 in patients with Neoplasm of colorectum
      * []
      * [u'https://www.ncbi.nlm.nih.gov/pubmed/10347231', u'https://www.ncbi.nlm.nih.gov/pubmed/9065403']
    * molecularmatch rectal neoplasm ['LGK']
      * CTNNB1 S37F confers sensitivity to Lgk-974 in patients with Neoplasm of colorectum
      * []
      * [u'https://www.ncbi.nlm.nih.gov/pubmed/10347231', u'https://www.ncbi.nlm.nih.gov/pubmed/11196159', u'https://www.ncbi.nlm.nih.gov/pubmed/11943721']